In [65]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from urllib.request import urlopen
import requests

# Initializing Driver and Loading Webpage

In [66]:
# Specify Options for Driver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
options.add_argument("--window-size=1920,1080")

# Instantiate Selenium Driver Object
driver = webdriver.Chrome(options=options)

# Load Webpage
url = 'https://www.scrapethissite.com/pages/frames/'
driver.get(url)

# Set Wait Object
wait = WebDriverWait(driver, 10)

In [67]:
# Wait Until iFrame is loaded
iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))

# Switch into the iFrame
driver.switch_to.frame(iframe)

In [68]:
data = []

# Initial Load of Turtle Cards, Used for Index in Inner Loop
turtles = driver.find_elements(By.CLASS_NAME, "turtle-family-card")

# Use Index of Turtle's List to Navigate through each card
for i in range(len(turtles)):

    # Reload the Cards after each iteration so they don't remain Stale
    turtles = driver.find_elements(By.CLASS_NAME, "turtle-family-card")

    # Find the Button For the Correct Turtle Based on Index and Click
    info_btn = turtles[i].find_element(By.TAG_NAME, "a")
    info_btn.click()

    # Once Inside the Additional Info iFrame, Wait for New info to Load
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "turtle-family-detail")))

    # Get the current webpage's HTML source and load into Soup Object
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract the Relevant Information
    name = soup.find(class_='family-name').get_text()
    info = soup.find(class_='lead').get_text(strip=True)

    # Store Info in Dictionary and append to Data List
    temp = {'Family' : name, 'Info' : info}
    data.append(temp)

    # Find 'Back to all Turtles' button and click on it
    back_button = driver.find_element(By.CLASS_NAME, 'turtle-family-detail').find_element(By.TAG_NAME, 'a')
    back_button.click()

    # Wait for the turtle cards again
    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "turtle-family-card")))


# Close Driver
driver.quit()

# Save and Store Data

In [69]:
# Convert to dataframe and export to csv 
pd.DataFrame(data).to_csv("turtles.csv", index=False)